# Example 2: Do Wealthier Countries Win More Medals?

## Tasks

* Examine correlation between
    * GDP and total medals
    * GDP per capita and medals per capita

* Segment countries into income tiers
* Compare Medals distributions acrosss tiers

* Pearson Correlation
* Spearman correlation (rank-based)

* Scatter plot with trendline
* Boxplots by income tier